In [48]:
import pandas as pd
import numpy as np

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from stop_words import get_stop_words

import string
import re
from collections import Counter
from collections import OrderedDict
import unidecode
import requests

In [14]:
data = pd.read_json('gg2015.json')
data = pd.DataFrame(data)

In [15]:
data = data['text'].values.tolist()

In [16]:
#print(data[:10])

In [62]:
people = set()

In [63]:
url = 'https://query.wikidata.org/sparql'
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
           wdt:P106/wdt:P279* wd:Q2526255; #uncomment for     FILM director (no award for TV director)
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
kb = r.json()
for item in kb['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [64]:
url = 'https://query.wikidata.org/sparql'
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
           wdt:P106/wdt:P279* wd:Q10800557; #uncomment for    FILM actor (don't just use actor)
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
kb = r.json()
for item in kb['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [65]:
url = 'https://query.wikidata.org/sparql'
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
           wdt:P106/wdt:P279* wd:Q10798782; #uncomment for    TV actor (don't just use actor)
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
kb = r.json()
for item in kb['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [66]:
url = 'https://query.wikidata.org/sparql'
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
           wdt:P106/wdt:P279* wd:Q36834; #uncomment for       composer (cannot use songwriter)
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
kb = r.json()
for item in kb['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [67]:
url = 'https://query.wikidata.org/sparql'
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
           wdt:P106/wdt:P279* wd:Q28389; #uncomment for       screenwriter
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
kb = r.json()
for item in kb['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [19]:
#print(tv_actors)

In [20]:
winners = {'cecil b. demille award': ['George', 'Clooney'],
    'best performance by an actress in a motion picture - drama': ['Julianne', 'Moore'],
    'best performance by an actor in a motion picture - drama': ['Eddie', 'Redmayne'],
    'best performance by an actress in a motion picture - comedy or musical': ['Amy', 'Adams'],
    'best performance by an actor in a motion picture - comedy or musical': ['Michael', 'Keaton'],
    'best performance by an actress in a supporting role in a motion picture': ['Patricia', 'Arquette'],
    'best performance by an actor in a supporting role in a motion picture': ['Simmons', 'J.', 'K.'],
    'best director - motion picture': ['Richard', 'Linklater'],
    'best screenplay - motion picture': ['Alejandro', 'Inarritu', 'Nicolas', 'Giacobone', 'Alexander', 'Dinelaris', 'Armando', 'Bo'],
    'best original score - motion picture': ['Johann', 'Johannsson'],
    'best performance by an actress in a television series - drama': ['Ruth', 'Wilson'],
    'best performance by an actor in a television series - drama': ['Kevin', 'Spacey'],
    'best performance by an actress in a television series - comedy or musical': ['Gina', 'Rodriguez'],
    'best performance by an actor in a television series - comedy or musical': ['Jeffrey', 'Tambor'],
    'best performance by an actress in a mini-series or motion picture made for television': ['Maggie', 'Gyllenhaal'],
    'best performance by an actor in a mini-series or motion picture made for television': ['Billy', 'Bob', 'Thorton'],
    'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television': ['Joanne', 'Froggatt'],
    'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television': ['Matt', 'Bomer']}

In [21]:
f = open('english.txt', 'r')
stop_words = f.read().splitlines()
#print(stopwords)

In [54]:
OFFICIAL_AWARDS_1315 = ['cecil b. demille award',
                        'best performance by an actress in a motion picture - drama', 
                        'best performance by an actor in a motion picture - drama',
                        'best performance by an actress in a motion picture - comedy or musical', 
                        'best performance by an actor in a motion picture - comedy or musical',
                        'best performance by an actress in a supporting role in a motion picture', 
                        'best performance by an actor in a supporting role in a motion picture', 
                        'best director - motion picture', 
                        'best screenplay - motion picture', 
                        'best original score - motion picture',
                        'best performance by an actress in a television series - drama', 
                        'best performance by an actor in a television series - drama',
                        'best performance by an actress in a television series - comedy or musical', 
                        'best performance by an actor in a television series - comedy or musical', 
                        'best performance by an actress in a mini-series or motion picture made for television', 
                        'best performance by an actor in a mini-series or motion picture made for television', 
                        'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television', 
                        'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television']

In [27]:
def parseAward(award):
    award = re.split('\W+', award)
    award = [i for i in award if i not in stop_words]
    return award

In [28]:
def getCategoriesDict():
    categories_dict = dict()
    for a in OFFICIAL_AWARDS_1315:
        terms = parseAward(a)
        categories_dict[a] = terms

    return categories_dict

In [29]:
categories_dict = getCategoriesDict()

In [30]:
def filter0(data, list1):
    result = []
    
    list1 = [i for i in list1 if i != 'performance' and i != 'role']

    for tweet in data:
        if all(term in tweet.lower() for term in list1):
            result.append(tweet)

    return result

In [31]:
def extractNominees(data, list1):
    #print(data[0])
    result = []
       
    #tt = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=True)
    translator = str.maketrans('', '', string.punctuation)
    remove_terms = ['#goldenglobes', 'golden globes', '#goldenglobe', 'golden globe', 'goldenglobes', 'goldenglobe', 'rt', 'golden', 'globe', 'globes']    
    stop = remove_terms + list1
    
    for tweet in data:
        #print(tweet)
        tweet = re.sub("\d+", "", tweet) #strip nums
        tweet = re.sub(r'http\S+', '', tweet) #strip urls
        tweet = re.sub(r'#\S+', '', tweet) #strip hashtags
        tweet = tweet.translate(translator) #strip non-alphanumeric characters
        tweet = tweet.split() #tokenize
        tweet = [term for term in tweet if term.lower() not in stop_words] #remove stop words
        for i in stop:
            for j in tweet:
                if i.lower() in j.lower():
                    tweet.remove(j)
        result.append(tweet)
        
    #print(result[:20])
        
    bgrams = [];

    for tweet in result:
        if tweet:
            bigram = list(nltk.bigrams(tweet))
            #print(bigram[:10])
            for bg in bigram:
                if bool(re.match(r'\b[A-Z][a-z]+\b', bg[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', bg[1])):
                    #print(bg[0], bg[1])
                    bgrams.append(bg)
            #tweet = re.findall('([A-Z][a-z]+)', tweet)
            #if tweet:
                #bgrams += list(nltk.bigrams(tweet))
  
    fdist = nltk.FreqDist(bgrams)
    #print(fdist)

    try:
        names = fdist.most_common()
        #names = [' '.join(i[0]) for i in fdist.most_common()]
    except:
        names = "nothing here"
    
    return names

In [68]:
def getNominees():
    d = {}
    for award in OFFICIAL_AWARDS_1315:
        d["{0}".format(award)] = filter0(data, categories_dict[award])
            
    nominees = {}
    for award in OFFICIAL_AWARDS_1315:
        nominees["{0}".format(award)] = extractNominees(d[award], categories_dict[award])
    
    return nominees

In [33]:
import timeit
start_time = timeit.default_timer()
getNominees()
elapsed = timeit.default_timer() - start_time
print(elapsed)

76.71186219300012


In [69]:
nominees = getNominees()

In [58]:
nominees.keys()

dict_keys(['cecil b. demille award', 'best performance by an actress in a motion picture - drama', 'best performance by an actor in a motion picture - drama', 'best performance by an actress in a motion picture - comedy or musical', 'best performance by an actor in a motion picture - comedy or musical', 'best performance by an actress in a supporting role in a motion picture', 'best performance by an actor in a supporting role in a motion picture', 'best director - motion picture', 'best screenplay - motion picture', 'best original score - motion picture', 'best performance by an actress in a television series - drama', 'best performance by an actor in a television series - drama', 'best performance by an actress in a television series - comedy or musical', 'best performance by an actor in a television series - comedy or musical', 'best performance by an actress in a mini-series or motion picture made for television', 'best performance by an actor in a mini-series or motion picture mad

In [70]:
def compareToKB(list1):
    for i in list1:
        j = ' '.join(i[0])
        if j in people:
            print(i)

In [ ]:
#Julianne Moore – Still Alice as Dr. Alice Howland - 3132
#Jennifer Aniston – Cake as Claire Bennett -13
#Felicity Jones – The Theory of Everything as Jane Hawking - 6
#Rosamund Pike – Gone Girl as Amy Elliott-Dunne - 11
#Reese Witherspoon – Wild as Cheryl Strayed - 6

In [71]:
compareToKB(nominees['cecil b. demille award'])

(('George', 'Clooney'), 2440)
(('Julianna', 'Margulies'), 30)
(('Liza', 'Minnelli'), 5)
(('Michael', 'Keaton'), 5)
(('Meryl', 'Streep'), 3)
(('Jim', 'Carrey'), 2)
(('Jodie', 'Foster'), 2)
(('Gary', 'Busey'), 1)
(('Pia', 'Zadora'), 1)
(('Kevin', 'Spacey'), 1)
(('Matt', 'Damon'), 1)
(('Dakota', 'Fanning'), 1)
(('Angelina', 'Jolie'), 1)
(('Richard', 'Linklater'), 1)


In [72]:
compareToKB(nominees['best performance by an actress in a motion picture - drama'])

(('Julianne', 'Moore'), 3132)
(('Jennifer', 'Aniston'), 13)
(('Rosamund', 'Pike'), 11)
(('Patricia', 'Arquette'), 9)
(('Eddie', 'Redmayne'), 7)
(('Reese', 'Witherspoon'), 6)
(('Felicity', 'Jones'), 6)
(('Shailene', 'Woodley'), 6)
(('Amy', 'Adams'), 4)


In [73]:
compareToKB(nominees['best performance by an actor in a motion picture - drama'])

(('Eddie', 'Redmayne'), 3041)
(('David', 'Oyelowo'), 13)
(('Benedict', 'Cumberbatch'), 10)
(('Jake', 'Gyllenhaal'), 7)
(('Michael', 'Keaton'), 6)
(('Julianne', 'Moore'), 6)
(('Gwyneth', 'Paltrow'), 6)
(('Steve', 'Carell'), 4)
(('Jennifer', 'Aniston'), 1)
(('George', 'Clooney'), 1)


In [74]:
compareToKB(nominees['best performance by an actress in a motion picture - comedy or musical'])

(('Amy', 'Adams'), 3262)
(('Patricia', 'Arquette'), 6)
(('Emily', 'Blunt'), 5)
(('Helen', 'Mirren'), 4)
(('Ricky', 'Gervais'), 4)
(('Julianne', 'Moore'), 3)
(('Quvenzhane', 'Wallis'), 1)
(('Meryl', 'Streep'), 1)
(('Lana', 'Parrilla'), 1)


In [75]:
compareToKB(nominees['best performance by an actor in a motion picture - comedy or musical'])

(('Michael', 'Keaton'), 2526)
(('Amy', 'Adams'), 5)
(('Ralph', 'Fiennes'), 3)
(('Joaquin', 'Phoenix'), 2)
(('Christoph', 'Waltz'), 2)
(('Jeffrey', 'Tambor'), 1)
(('Jamie', 'Foxx'), 1)
(('Wes', 'Anderson'), 1)


In [76]:
compareToKB(nominees['best director - motion picture'])

(('Richard', 'Linklater'), 1141)
(('Ellar', 'Coltrane'), 14)
(('Wes', 'Anderson'), 5)
(('David', 'Fincher'), 2)
(('Ethan', 'Hawke'), 1)
(('Patricia', 'Arquette'), 1)
(('Eddie', 'Redmayne'), 1)


In [59]:
compareToKB(nominees['best screenplay - motion picture'])

(('Kristen', 'Wiig'), 1)
(('Emma', 'Stone'), 1)
(('Michael', 'Keaton'), 1)


In [43]:
compareToKB(nominees['best original score - motion picture'])

(('Sienna', 'Miller'), 2)
(('Vince', 'Vaughn'), 2)
(('John', 'Legend'), 2)
(('Emma', 'Stone'), 1)
(('Kevin', 'Spacey'), 1)


In [44]:
compareToKB(nominees['best performance by an actress in a television series - drama'])

(('Viola', 'Davis'), 5)
(('Claire', 'Danes'), 4)
(('Lana', 'Parrilla'), 4)
(('Julianna', 'Margulies'), 2)
(('Robin', 'Wright'), 1)


In [82]:
# RUTH WILSON NOT IN KB EVEN THOUGH SHE IS AN ACTOR AND ALIVE

In [79]:
compareToKB(nominees['best performance by an actor in a television series - drama'])

(('Kevin', 'Spacey'), 2270)
(('Frank', 'Underwood'), 3)
(('Clive', 'Owen'), 2)
(('Liev', 'Schreiber'), 2)
(('Dominic', 'West'), 2)
(('James', 'Spader'), 1)


In [83]:
compareToKB(nominees['best performance by an actress in a television series - comedy or musical'])

(('Gina', 'Rodriguez'), 10)


In [84]:
compareToKB(nominees['best performance by an actor in a television series - comedy or musical'])

(('Jeffrey', 'Tambor'), 738)
(('Ricky', 'Gervais'), 2)


In [90]:
compareToKB(nominees['best performance by an actress in a mini-series or motion picture made for television'])

(('Maggie', 'Gyllenhaal'), 4)


In [91]:
nominees['best performance by an actress in a mini-series or motion picture made for television']

[(('Joanne', 'Froggatt'), 4),
 (('Froggatt', 'Downton'), 4),
 (('Maggie', 'Gyllenhaal'), 4),
 (('Downton', 'Abbe'), 3),
 (('Downton', 'Abbey'), 1),
 (('Gyllenhaal', 'Honorable'), 1),
 (('Honorable', 'Woman'), 1)]

In [89]:
compareToKB(nominees['best performance by an actor in a mini-series or motion picture made for television'])

(('Matt', 'Bomer'), 4)


In [94]:
nominees['best performance by an actor in a mini-series or motion picture made for television']

[(('Bob', 'Thornton'), 66),
 (('Billy', 'Bob'), 64),
 (('Thornton', 'Fargo'), 10),
 (('Matt', 'Bomer'), 4),
 (('Bomer', 'Normal'), 3),
 (('Fargo', 'Fargo'), 1),
 (('Congrats', 'Billy'), 1),
 (('Role', 'Matt'), 1),
 (('Role', 'Matt…'), 1),
 (('Bob', 'Fargo'), 1)]

In [92]:
compareToKB(nominees['best performance by an actress in a supporting role in a series, mini-series or motion picture made for television'])

In [93]:
compareToKB(nominees['best performance by an actor in a supporting role in a series, mini-series or motion picture made for television'])

(('Matt', 'Bomer'), 4)


In [ ]:
#Eddie Redmayne – The Theory of Everything as Stephen Hawking - 3041
#Steve Carell – Foxcatcher as John du Pont - 4
#Benedict Cumberbatch – The Imitation Game as Alan Turing - 10
#Jake Gyllenhaal – Nightcrawler as Louis "Lou" Bloom - 7
#David Oyelowo – Selma as Martin Luther King Jr. - 13

In [ ]:
c2 = filter0(data, categories_dict['best performance by an actor in a motion picture - drama'])  
actor_movie_drama = extractNominees(c2, categories_dict['best performance by an actor in a motion picture - drama'])

In [ ]:
compareToKB(actor_movie_drama)

In [ ]:
#Amy Adams – Big Eyes as Margaret Keane - 3262
#Emily Blunt – Into the Woods as The Baker's Wife - 5
#Helen Mirren – The Hundred-Foot Journey as Madame Mallory - 4
#Julianne Moore – Maps to the Stars as Havana Segrand - 3
#Quvenzhané Wallis – Annie as Annie Bennett - 3

In [ ]:
c3 = filter0(data, categories_dict['best performance by an actress in a motion picture - comedy or musical'])  
actress_movie_cm = extractNominees(c3, categories_dict['best performance by an actress in a motion picture - comedy or musical'])

In [ ]:
compareToKB(actress_movie_cm)

In [ ]:
#conclusion: for 2015 data, searching for all terms at once about 10s slower, about same accuracy
#although search all terms may be signicantly slower if include other ngrams
#both get J.K. Simmons wrong -> Simmons Whiplash
#search all terms gets nothing for supporting actress in series, mini series, or motion picture ('Joanne Froggatt')
#search all terms gets 'the theory' for best original score
#hardcoded filtering gets 'the theory' for best original score
#hardcoded filtering gets 'the theory' for best actor motion picture drama (eddie redmayne)

In [ ]:
c12 = filter0(data, categories_dict['best performance by an actor in a television series - drama'])
n = extractNominees(c12, categories_dict['best performance by an actor in a television series - drama'])

In [ ]:
for i in n:
    if i in people:
        print(i)
        

In [ ]:
if 'Clive Owen' in tv_actors:
    print(True)

In [104]:
bool(re.search(r'(\b[A-Z][a-z]+\b)(.*)(\b[A-Z][a-z]+\b)*', 'I really like the movie Life of Pie'))

True